<a href="https://colab.research.google.com/github/berrygayo/Python_Study/blob/master/%EC%B0%A8%EC%99%80_%EB%B9%84%ED%96%89%EA%B8%B0_%EC%9D%B4%EB%AF%B8%EC%A7%80_%EB%B6%84%EB%A5%98%ED%95%98%EA%B8%B0v0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 차와 비행기 이미지 분류하기 
참고 : 
https://codetorial.net/tensorflow/classifying_the_cats_and_dogs.html

In [ ]:
# tensorflow와 tf.keras 임포트 
import tensorflow as tf 
from tensorflow import keras

# 헬퍼(helper) 라이브러리 임포트 
import numpy as np
import matplotlib.pyplot as plt 

print(tf.__version__)

In [ ]:
# 데이터셋 불러오기 
!unzip '/content/drive/My Drive/dataset/cars_and_plane2.zip'

In [ ]:
from IPython.display import Image
Image('/content/cars_and_plane2/training_set/train_cars/Acura_MDX_2014_42_18_290_35_6_77_67_193_20_FWD_7_4_SUV_pso.jpg')

# 경로 지정하기 

In [ ]:
import os
# 기본 경로
base_dir = '/content/cars_and_plane2'

train_dir = os.path.join(base_dir, 'training_set')
validation_dir = os.path.join(base_dir, 'validation_set')

# 훈련에 사용되는 이미지 경로
train_cars_dir = os.path.join(train_dir, 'train_cars')
train_plane_dir = os.path.join(train_dir, 'train_plane')
print(train_cars_dir)
print(train_plane_dir)

# 테스트에 사용되는 이미지 경로
validation_cars_dir = os.path.join(validation_dir, 'vali_cars')
validation_plane_dir = os.path.join(validation_dir, 'vali_plane')
print(validation_cars_dir)
print(validation_plane_dir)

# 파일 이름과 개수 

In [ ]:
train_cars_fnames = os.listdir( train_cars_dir ) 
#os.listdir() 메서드는 경로 내에 있는 파일의 이름을 리스트로 만든다. 
train_plane_fnames = os.listdir( train_plane_dir )

print(train_cars_fnames[:5])
print(train_plane_fnames[:5])

In [ ]:
# 각 경로에 있는 파일명 리스트의 길이를  통해 파일의 개수 확인
print('Total training cars images :', len(os.listdir(train_cars_dir)))
print('Total training plane images :', len(os.listdir(train_plane_dir)))

print('Total validation cars images :', len(os.listdir(validation_cars_dir)))
print('Total validation plane images :', len(os.listdir(validation_plane_dir)))

# 이미지 확인하기 

In [ ]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

nrows, ncols = 4, 4
pic_index = 0

fig = plt.gcf()
fig.set_size_inches(ncols*3, nrows*3)

pic_index+=8

next_cars_pix = [os.path.join(train_cars_dir, fname)
                for fname in train_cars_fnames[ pic_index-8:pic_index]]

next_plane_pix = [os.path.join(train_plane_dir, fname)
                for fname in train_plane_fnames[ pic_index-8:pic_index]]

for i, img_path in enumerate(next_cars_pix+next_plane_pix):
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off')

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

# 모델 구성하기 

In [ ]:
import tensorflow as tf


model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

# 모델 컴파일하기


In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=0.001),
            loss='binary_crossentropy',
            metrics = ['accuracy'])

# 이미지 데이터 전처리하기
훈련을 진행하기 전, tf.keras.preprocessing.image 모듈의 ImageDataGenerator 클래스를 이용해서 데이터 전처리를 진행합니다.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

train_generator = train_datagen.flow_from_directory(train_dir,
                                                  batch_size=20,
                                                  class_mode='binary',
                                                  target_size=(150, 150))

validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                       batch_size=20,
                                                       class_mode  = 'binary',
                                                       target_size = (150, 150))

# 모델 훈련하기 

In [ ]:
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    steps_per_epoch=10, #한번에 학습시킬 training 데이터 batch 수 
                    epochs=10,
                    validation_steps=10, # 한번에 학습시킬 validation 데이터 batch 수  
                    verbose=2)

# 정화도와 손실 확인하기 

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.plot(epochs, val_acc, 'bo', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'g', label='Training Loss')
plt.plot(epochs, val_loss, 'go', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

# 테스트 이미지 분류하기 



In [ ]:
Image('/content/cars_and_plane2/test_set/test_cars/car1.jpg')

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded=files.upload()

for fn in uploaded.keys():

  path='/content/' + fn
  img=image.load_img(path, target_size=(150, 150))

  x=image.img_to_array(img)
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])

  classes = model.predict(images, batch_size=10)

  print(classes[0])

  if classes[0]>0:
    print(fn + " is a plane")
  else:
    print(fn + " is a car")

In [ ]:
Image('/content/cars_and_plane2/test_set/test_plane/plane1.jpg')

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded=files.upload()

for fn in uploaded.keys():

  path='/content/' + fn
  img=image.load_img(path, target_size=(150, 150))

  x=image.img_to_array(img)
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])

  classes = model.predict(images, batch_size=10)

  print(classes[0])

  if classes[0]>0:
    print(fn + " is a plane")
  else:
    print(fn + " is a car")

In [ ]:
Image('/content/drive/My Drive/dataset/test_flyingcar_1.jpg')

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded=files.upload()

for fn in uploaded.keys():

  path='/content/' + fn
  img=image.load_img(path, target_size=(150, 150))

  x=image.img_to_array(img)
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])

  classes = model.predict(images, batch_size=10)

  print(classes[0])

  if classes[0]>0:
    print(fn + " is a plane")
  else:
    print(fn + " is a car")

In [ ]:
# 확률 나올 수 있도록 모델 재설정 
model2= keras.Sequential([
                          keras.layers.Flatten(input_shape=(150,150)), #150*150 픽셀의 이미지 포멧을 1차원 배열로 변환 
                          keras.layers.Dense(128, activation='relu'),
                          keras.layers.Dense(2, activation='softmax')  # 두 개 클래스 중 하나에 속할 확률 출력                      
])

In [ ]:
model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model2.fit(train_images, train_labels, epochs=5)